# Paradas de autobuses en Rosario
<p>Aplicación basada en mis estudios del curso <a href=#>Python for Data Science</a> de la Universidad de California: San Diego<br />
Muestra en un mapa de la ciudad de Rosario las paradas de autobuses, permite escoger una línea de autobuses para ver su recorrido de ida y vuelta en el mapa.</p>

## Datasets
Se usan los datos suministrados por la municipalidad de Rosario en su plataforma de datos abiertos <a href=https://datos.rosario.gob.ar>RosarioDatos</a>

In [1]:
import pandas as pd
import numpy as np
import folium
from folium.features import DivIcon
import ipywidgets as widgets
from ipywidgets import interact, interactive
from IPython.display import HTML

### Extracción de los datos relevantes
<p>Se carga el fichero csv y se muestran las propiedades. Luego se hace una lista de propiedades para dejar las columnas apropiadas para el trabajo</p>

In [2]:
# carga de datos de un csv
paradas = pd.read_csv('./data/paradas_tup_json.csv')

In [3]:
# propiedades de los datos
print(type(paradas))
print(paradas.shape)
paradas.columns

<class 'pandas.core.frame.DataFrame'>
(10010, 20)


Index(['SE_ROW_ID', 'MSLINK', 'PARADA', 'RAMAL', 'SENTIDO', 'ORDEN',
       'ID_PARADA', 'CALLE_UNO', 'CALLE_DOS', 'COD_SMS', 'OCHAVA', 'REFUGIO',
       'DISTRITO', 'PUNTO_X', 'PUNTO_Y', 'CAMBIO', 'USUARIO_CA', 'TYPE',
       'CHECKSUM', 'GEOJSON'],
      dtype='object')

In [4]:
# muestra de los datos en el fichero de paradas
paradas.head(3)

,SE_ROW_ID,MSLINK,PARADA,RAMAL,SENTIDO,ORDEN,ID_PARADA,CALLE_UNO,CALLE_DOS,COD_SMS,OCHAVA,REFUGIO,DISTRITO,PUNTO_X,PUNTO_Y,CAMBIO,USUARIO_CA,TYPE,CHECKSUM,GEOJSON
0,727,0,3152,45,V,0,3152,1077,1930,9776,NE,P,NOROESTE ...,-60.759785,-32.940745,2014-10-06,4,0,0,"{ ""type"": ""Feature"", ""properties"": { ""SE_ROW_I..."
1,748,0,2924,58,V,399,2924,12,1815,4300,N,I,NORTE ...,-60.699092,-32.873835,2010-11-17,2,0,0,"{ ""type"": ""Feature"", ""properties"": { ""SE_ROW_I..."
2,859,0,2983,56,I,23,2983,1412,1474,9163,NO,P,NORTE ...,-60.691754,-32.892053,2010-11-19,2,0,0,"{ ""type"": ""Feature"", ""properties"": { ""SE_ROW_I..."


In [5]:
# lista de propiedades a usar
# se renombra por etiquetas más claras
# se reordenan las columnas también por claridad
propiedades = ['ID_PARADA', 'RAMAL', 'SENTIDO', 'PUNTO_X', 'PUNTO_Y']
paradas_df = paradas[propiedades]
paradas_df = paradas_df.rename(index=str, columns={"PUNTO_X": "LONGITUD", "PUNTO_Y": "LATITUD"})
paradas_df = paradas_df[['ID_PARADA', 'RAMAL', 'SENTIDO', 'LATITUD', 'LONGITUD']]
paradas_df.head(3)

,ID_PARADA,RAMAL,SENTIDO,LATITUD,LONGITUD
0,3152,45,V,-32.940745,-60.759785
1,2924,58,V,-32.873835,-60.699092
2,2983,56,I,-32.892053,-60.691754


In [6]:
# se cargan los datos de los recorridos
# donde figura cada línea el ramal que tiene asignado
recorridos = pd.read_csv('./data/recorridos_tup_json.csv')
print(type(recorridos))
print(recorridos.shape)
recorridos.columns

<class 'pandas.core.frame.DataFrame'>
(136, 15)


Index(['SE_ROW_ID', 'MSLINK_ODB', 'LINEA', 'BANDERA', 'SENTIDO', 'ID_TUP',
       'LINEA_BANDERA', 'TIPO', 'FECHA_HORA', 'CHECKSUM', 'OBSERVACIONES',
       'HORARIO1', 'HORARIO2', 'ID_RAMAL', 'GEOJSON'],
      dtype='object')

In [7]:
# se unen el dataframe de paradas con las líneas de autobuses
# luego se filtrarán las paradas de acuerdo a la línea
paradas_df = paradas_df.merge(recorridos[['ID_RAMAL', 'LINEA_BANDERA']], left_on= 'RAMAL', right_on= 'ID_RAMAL' )
del paradas_df['ID_RAMAL']
paradas_df.head(3)

,ID_PARADA,RAMAL,SENTIDO,LATITUD,LONGITUD,LINEA_BANDERA
0,3152,45,V,-32.940745,-60.759785,142 Negra
1,3152,45,V,-32.940745,-60.759785,142 Negra
2,1445,45,V,-32.940207,-60.717191,142 Negra


In [8]:
# se crea una lista con todos los nombres de las líneas
lista_lineas = recorridos['LINEA_BANDERA'].unique()
lista_lineas.sort()

In [9]:
# para hacer más rápida la interacción al dibujar el mapa
# se separan las paradas de cada línea en un diccionario
# se guarda también el promedio de la latitud y longitud
# para mostrar el mapa centrado
layers = {}
for l in lista_lineas:
    mask_ramal = paradas_df['LINEA_BANDERA'] == l
    stage = paradas_df[mask_ramal][['LATITUD', 'LONGITUD', 'SENTIDO']]
    lat_mean= stage['LATITUD'].mean()
    long_mean= stage['LONGITUD'].mean()
    layers.update({l: [stage, lat_mean, long_mean]})

In [10]:
# leyenda para el mapa
def add_legend(m):
    legend_html =   '''
                <div style="position: fixed; 
                            top: 50px; right: 50px; padding:5px; 
                            background-color:#E7EFE888;
                            color:#505e62; z-index:9999; font-size:14px;">
                              &nbsp;<i class="fa fa-circle fa-lg" style="color:#EE8601"></i> Ida &nbsp; <br>
                              &nbsp;<i class="fa fa-circle fa-lg" style="color:#00ACEC"></i> Vuelta &nbsp; 
                </div>
                ''' 
         
    m.get_root().html.add_child(folium.Element(legend_html))

In [11]:
# se crea el mapa y se agregan los marcadores
def f(Linea):
    m = folium.Map(
        location=(layers[Linea][1], layers[Linea][2]),
        tiles='cartodbpositron',
        zoom_start=13
        )
    add_legend(m)
    stage = layers[Linea][0]
    for i, r in stage.iterrows():
        color_sentido= '#EE8601' if r['SENTIDO'] == 'I' else '#00ACEC'
        folium.CircleMarker([r['LATITUD'], r['LONGITUD']], 
                            radius=5,
                            weight=1,
                            color=color_sentido,
                            fill=True,
                            fillOpacity=0.01).add_to(m)
    display(m)

In [12]:
%%javascript
// se desactiva el autoscroll
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [13]:
# se agrega un widget para mostrar
w = interactive(f, Linea = lista_lineas)
display(w)

interactive(children=(Dropdown(description='Linea', options=('101 Negra', '101 Roja', '102 Negra', '102 Roja',…

<img src='imgs/maprender.png' />